# DEVoca-scraper: 단어 번역

- 스크래핑 한 단어와 설명을 [인공지능 번역 API](https://www.deepl.com/docs-api)를 활용해 한국어로 번역

In [3]:
import json
import os
import requests
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

True

In [2]:
URL = 'https://api-free.deepl.com/v2/translate'


In [8]:
word_data = []

with open('word_wiki.json', 'r', encoding='utf-8') as f:
    word_data = json.load(f)

word_data[:3]

[{'word_name_en': 'abstract data type (ADT)',
  'word_def': 'A mathematical model for data types in which a data type is defined by its behavior (semantics) from the point of view of a user of the data, specifically in terms of possible values, possible operations on data of this type, and the behavior of these operations. This contrasts with data structures, which are concrete representations of data from the point of view of an implementer rather than a user.'},
 {'word_name_en': 'abstract method',
  'word_def': 'One with only a signature and no implementation body. It is often used to specify that a subclass must provide an implementation of the method. Abstract methods are used to specify interfaces in some computer languages.'},
 {'word_name_en': 'abstraction',
  'word_def': '1.\xa0\xa0In software engineering and computer science, the process of removing physical, spatial, or temporal details or attributes in the study of objects or systems in order to more closely attend to other

- 요청 문서
  - https://www.deepl.com/ko/docs-api/translate-text/translate-text

In [4]:
def req_deepl(texts):
    response = requests.post(
        url=URL,
        headers={
            "Authorization": f"DeepL-Auth-Key {os.getenv('DEEPL_API_KEY')}",
        },
        data={
            "text": texts,
            "source_lang": "EN",
            "target_lang": "KO",
            "context": "Translating some computer science keywords and their definitions from English to Korean."
        }
    )

    # pprint(response.json(), sort_dicts=False)
    return response.json()['translations']

In [24]:
texts = [
    "In multitasking computer operating systems, a daemon (/\u02c8di\u02d0m\u0259n/ or /\u02c8de\u026am\u0259n/) is a computer program that runs as a background process, rather than being under the direct control of an interactive user. Traditionally, the process names of a daemon end with the letter d, for clarification that the process is in fact a daemon, and for differentiation between a daemon and a normal computer program. For example, syslogd is a daemon that implements system logging facility, and sshd is a daemon that serves incoming SSH connections."
]

req_deepl(texts)

{'translations': [{'detected_source_language': 'EN',
                   'text': '멀티태스킹 컴퓨터 운영 체제에서 데몬(/ˈdiːmən/ 또는 /ˈdeɪmən/)은 대화형 '
                           '사용자의 직접적인 제어를 받지 않고 백그라운드 프로세스로 실행되는 컴퓨터 프로그램입니다. '
                           '일반적으로 데몬의 프로세스 이름은 문자 d로 끝나는데, 이는 해당 프로세스가 실제로 '
                           '데몬임을 명확히 하고 데몬과 일반 컴퓨터 프로그램을 구분하기 위한 것입니다. 예를 들어, '
                           'syslogd는 시스템 로깅 기능을 구현하는 데몬이고, sshd는 들어오는 SSH 연결을 '
                           '서비스하는 데몬입니다.'}]}


In [7]:
translated_word_data = []

split_size = 50

word_data_splited = [word_data[i:i+split_size]
                     for i in range(0, len(word_data), split_size)]

for splited in word_data_splited:
    texts = []
    for word in splited:
        texts.append(word['word_name_en'])
        texts.append(word['word_def'])

    response_data = req_deepl(texts)

    for i in range(0, len(response_data), 2):
        dt = response_data[i]
        dd = response_data[i + 1]
        translated_word_data.append({
            'word_name_kr': dt['text'],
            'word_def': dd['text']
        })

In [11]:
word_translated_link = []

for word, translated in zip(word_data, translated_word_data):
    word_translated_link.append({
        'word_name_kr': translated['word_name_kr'],
        'word_name_en': word['word_name_en'],
        'word_def': translated['word_def'],
        'word_link': word['word_link']
    })

pprint(word_translated_link[:3], sort_dicts=False)

[{'word_name_kr': '추상 데이터 유형(ADT)',
  'word_name_en': 'abstract data type (ADT)',
  'word_def': '데이터 유형이 데이터 사용자 관점에서의 동작(의미론)에 의해 정의되는 데이터 유형에 대한 수학적 모델로, 특히 '
              '가능한 값, 이 유형의 데이터에 대해 가능한 연산 및 이러한 연산의 동작 측면에서 정의됩니다. 이는 사용자가 아닌 '
              '구현자의 관점에서 데이터를 구체적으로 표현한 데이터 구조와는 대조적입니다.',
  'word_link': 'https://en.wikipedia.org/wiki/Abstract_data_type'},
 {'word_name_kr': '추상 메서드',
  'word_name_en': 'abstract method',
  'word_def': '서명만 있고 구현 본문이 없는 메서드. 서브클래스가 메서드의 구현을 제공해야 한다는 것을 지정하는 데 자주 '
              '사용됩니다. 추상 메서드는 일부 컴퓨터 언어에서 인터페이스를 지정하는 데 사용됩니다.',
  'word_link': 'https://en.wikipedia.org/wiki/Abstract_method'},
 {'word_name_kr': '추상화',
  'word_name_en': 'abstraction',
  'word_def': '1.  소프트웨어 공학 및 컴퓨터 과학에서 사물이나 시스템을 연구할 때 물리적, 공간적 또는 시간적 세부 사항이나 '
              '속성을 제거하여 다른 관심 있는 세부 사항에 더 집중하는 과정으로, 일반화 과정과도 본질적으로 매우 유사합니다.',
  'word_link': 'https://en.wikipedia.org/wiki/Abstraction_(software_engineering)'}]


In [12]:
with open("word_translated_link.json", "w") as f:
    json.dump(word_translated_link, f, indent=2)

- 사용량 확인

In [13]:
pprint(
    requests.get(
        url="https://api-free.deepl.com/v2/usage",
        headers={
            "Authorization": f"DeepL-Auth-Key {os.getenv('DEEPL_API_KEY')}",
        }
    ).json(),
    sort_dicts=False
)

{'character_count': 380734, 'character_limit': 500000}
